In [1]:
!nvidia-smi

Mon Oct  7 09:32:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
path = '/content/drive/MyDrive/DACON/Finance/reprocessed/'
path = '/content/drive/MyDrive/kdt-EST-AI/project/dacon_fis/src/'
base_directory = path # Your Base Directory

In [3]:
dataset_name = "kdt3/DACON-QA-large-ensemble-markdown-reprocessed-512"
train_name = "kdt3/DACON-QA-large-ensemble-markdown-newprompt-512"
fname = "gemma2_large_ensemble_markdown_512_3epoch_refactored.csv"

# 설명

## Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

## Mount/Login

구글 드라이브를 마운트하고 허깅페이스에 로그인
- 이때 허깅페이스 토큰은 kdt3 그룹에 대해 읽기/쓰기 권한이 있는 토큰이어야 함

## Download Library
필요/사용 라이브러리 다운로드
이때 버전 문제로 설치를 한 뒤 세션을 한번 재시작해줘야 합니다
<br>(그리고 세션 완전히 끊기면 다운로드 후 재시작을 다시 해줘야...)

## Import Library
한번 재시작했으면 위 과정 없이 Import만 실행해주면 됩니다

## Vector DB
문서를 여러 조각(chunk)로 나누고, 임베딩 유사도를 통해 관련 조각을 찾을 수 있게 DB화하는 함수들이 정의되어 있습니다.

## DB 생성
Vector DB에서 정의된 함수들로 문서 DB를 만들어줍니다.<br><br>
이때 Train과 Test를 한번에 하려고 하면 코랩이 터질 확률이 높으므로 Train하고 Create Dataset까지 실행해 업로드 한 뒤 재시작해서 램을 비우고 Test를 하는 것이 좋습니다.<br> 또한 문서 임베딩을 어떤 모델로 할지 인자로 넘겨줄 수 있습니다

## Create Dataset
DB 생성에서 만든 db와 데이터 dataframe을 사용해 HuggingFace 데이터셋 생성 후 업로드

## Fine-Tuning
학습 데이터셋으로 모델에 대한 파인튜닝 진행 후 Huggingface에 업로드<br>
4비트 양자화 LoRA로 파인튜닝<br>
기반 모델 또는 넣어줄때 사용할 프롬프트, 학습 관련 하이퍼파라미터 수정 가능

## Langchain 을 이용한 추론
모델을 사용한 추론


## 실행
### 기본
Mount/Login -> Download Library -> 재시작 (처음 1번)
Mount/Login -> Import Library (이후)

### 데이터셋 만들기
기본 -> Vector DB -> DB 생성 -> Create Dataset에서 첫 셀 + Train/Valid/Test 중 해당하는 셀

### 모델 학습하기
기본 -> Fine-Tuning(업로드할 위치, 데이터셋 위치, 모델 링크 확인 필수)

### 학습된 모델로 추론하기
기본 -> Langchain을 이용한 추론(모델 링크, 데이터셋 위치 확인) -> Submission(저장할 파일명 확인)

# Mount/Login

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os

token_path = os.path.join(base_directory,'data','token')
with open(token_path,'r') as f:
    hf_token = f.readline().strip('\n')
    wandb_token = f.readline().strip('\n')

In [6]:
from huggingface_hub import login

login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
!pip install wandb
import wandb
# wandb 개인 API 키 입력
wandb.login(key=wandb_token)

import os

os.environ["WANDB_PROJECT"]="DACON_FinAI"
#os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb_run_name="noaug-gemma2-lora-large"

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: doeun-235 (DACON-FinAI). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Download Library

In [8]:
!apt-get install tesseract-ocr
!apt-get install poppler-utils

!pip install orjson==3.10.6

!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install langchain-teddynote
!pip install sentence-transformers
!pip install faiss-gpu
!pip install peft
!pip install trl
!pip install -U accelerate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple/


# Import Library

In [9]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm.auto import tqdm

from langchain.document_loaders.parsers.pdf import PDFPlumberParser

In [10]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# peft
from peft import prepare_model_for_kbit_training
from peft import PeftModel
from peft import LoraConfig, get_peft_model


# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

# PDF 로딩/청크화 관련
from langchain.document_loaders.parsers.pdf import PDFPlumberParser
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever


/usr/local/lib/python3.10/dist-packages/langchain_teddynote/retrievers/__init__.py:14: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .kiwi_bm25 import KiwiBM25Retriever


# Fine-Tuning

In [11]:
# 모델 ID
model_cands={
 'llama2' : "beomi/llama-2-ko-7b",
 'yi' : "beomi/Yi-Ko-6B",
 'solar-beom' : "beomi/Solar-Ko-Recovery-11B",
 'gemma2' : "rtzr/ko-gemma-2-9b-it",
 'solar-lee' : "chihoonlee10/T3Q-ko-solar-dpo-v8.0",
 'llama31' : "KISTI-KONI/KONI-Llama3-8B-Instruct-20240729",
}

model_id = model_cands['gemma2']

In [12]:
# 모델 로드 및 양자화 설정 적용

def load_model_w_setting(model_id,add_output_token=False,**kwargs):
  # 4비트 양자화 설정
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  # 토크나이저 로드 및 설정
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.use_default_system_prompt = False
  tokenizer.padding_side="right"

  model= AutoModelForCausalLM.from_pretrained(
      model_id,
      quantization_config=bnb_config,
      device_map="auto",
      trust_remote_code=True,
      **kwargs
      )

# 일부 모델의 경우 토크나이저에 답변 토큰 추가 작업 필요
  if add_output_token :
    initial_token_count = len(tokenizer)
    response_template = '답변: '
    added_token_count = tokenizer.add_special_tokens({"additional_special_tokens": [response_template]})
    model.resize_token_embeddings(new_num_tokens=initial_token_count+added_token_count)

  return model, tokenizer

In [13]:
model,tokenizer = load_model_w_setting(model_id,attn_implementation='eager')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
#데이터셋 로드
from datasets import load_dataset

dataset_url = dataset_name
train_dataset = load_dataset(dataset_url)

README.md:   0%|          | 0.00/544 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/262k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/133k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/496 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/98 [00:00<?, ? examples/s]

In [15]:
# 문맥 잘못됐나 확인 - 문맥 길이 체크
import numpy as np
amax = np.argmax([len(x) for x in train_dataset['train']['context']])
amax, len(train_dataset['train']['context'][amax])

(179, 3965)

In [16]:
def print_dataset_ele(data,i):
  print(data['question'][i])
  print('--------')
  print(data['context'][i])
  print('--------')
  print(data['answer'][i])

#예시
i= amax
print_dataset_ele(train_dataset['train'],i)

국가채무가 과다할 경우 어떤 위험이 발생할 수 있는가?
--------
금주의는 현금의 수
입과 지출이 일어난  
5 경제활동이 발생한 다른 경우 재무정보의 신속성과 False
시점과 현금거래 시 정확성이
점이 부족하다는 단
점  
6 발생주의는 실제경 장부에 기록되므로 신속한 재무정 False
제활동이 발생한 시 보  
7 현금주의발생주의
동결과를어떤자산 하기
으로 인식하는가에
어렵다는 단점  
8 현금주의 발생주의 False  
9 - 현금의 수입과 지출 - 현금의 변동이외 권리·의무의 변 False
시점에 회계처리 •현 동을 포함한 경제적 실질이 변동
금흐름이 발생하는 하는 시점에 회계처리 •현금의 유·
사건(현금의 증감)의 출입이 없더라도 자산 및 부채 평
결과만 기록 가로 경제적 자원 변동 결과 기록  
-----  
###### 03-3 공공부문부채(D3)  
2016년 2017년  
1,036.6 [조원] 1,044.6[조원]  
≆ 공공부문부채(D3)는 국가간 공공부문 재정건전성 관리에 활용되는 지표로, 중앙
하는 채권  
-  차입금은 정부가 한국은행, 민간기관 또는 국제기금 등으로부터 법정 유가증권 발행  
없이 차입하는 금액  
-  국고채무부담행위는 국가가 국회의 사전의결을 받은 범위 내에서 예산의 확보 없이  
미리 채무를 부담하는 행위  
국가의 회계 또는 기금은 중앙관서의 장이 관리·운용하는 회계 또는 기금을 말하며,  
중앙관서의 장이 관리·운용하지 않는 공공기관 관리기금[7]과 지방정부 순채무[8]는  
제외  
금전채무는 국가의 회계 또는 기금이 발행한 채권, 차입금, 국고채무부담행위와  
국가보증채무[9] 중 정부가 대신 채무를 이행하기로 확정된 채무를 포함  
-  정부가 주 채무자 대신 채무를 이행하기로 확정되지 않은 국가보증채무는 국가채무  
에서 제외  
**7  공무원연금기금, 사립학교교직원연금기금, 중소기업진흥 및 산업기반기금, 기술신용보증기금, 국민체육진흥기금 등**  
**8  「국가재정법」은 중앙정부의 재정을 

In [17]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:498: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [18]:
# 학습 프롬프트 - 추론 프롬프트와 통일하는 것이 좋다고 함
new_template ='''"task_instructions" : [

당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 세 문장 이내로 답변하세요.
 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.
 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.
 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.
 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장들로 답변하세요.
 6. 최대한 간략한 내용으로, 최대한 적은 개수의 문장으로 답변하세요.
 7. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, '답변: ', '참고: '를 절대로 출력하지 마세요.

]

"context":
{context},

"question":
{question},

"주어진 질문에 대한 답변만 한 문장으로 생성한다."

"answer":
{answer}<eos>'''

template ='''"task_instructions" : [

당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요.
 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.
 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.
 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.
 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장들로 답변하세요.
 6. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, '답변: ', '참고: '를 절대로 출력하지 마세요.

]

"context":
{context},

"question":
{question},

"주어진 질문에 대한 답변만 한 문장으로 생성한다."

"answer":
{answer}<eos>'''

response_template = '"answer":\n'

def formatting_prompts_func(example, template=template):
    output_texts = []
    for i in range(len(example['question'])):
        context = example['context'][i]
        question = example['question'][i]
        answer = example['answer'][i]
        output_texts.append(template.format(context=context,question=question,answer=answer))
    return output_texts



In [19]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import collections
# from SQUAD, 메트릭 계산 함수

def compute_f1(y_true, y_pred):
    common = collections.Counter(y_true) & collections.Counter(y_pred)
    num_same = sum(common.values())
    precision = 1.0 * num_same / len(y_pred) if len(y_pred) != 0 else 0
    recall = 1.0 * num_same / len(y_true) if len(y_true) != 0 else 0
    if len(y_pred) == 0 or len(y_true) == 0: f1 = int(y_pred == y_true)
    elif num_same == 0: f1 = 0
    else: f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1, precision, recall = compute_f1(labels,preds)
#    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc
    }

In [20]:
import transformers, os
from datetime import datetime
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# 로컬에 모델 저장하고 싶은 경우 이름 지정
project = "financeQA-finetune"
base_model_name = "gemma2"
run_name = base_model_name + "_" + project
output_dir = os.path.join(path,run_name)
if not os.path.exists(output_dir) : os.makedirs(output_dir)

MAX_LEN = 2048 * 2

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset['train'],
    eval_dataset = train_dataset['valid'],
    compute_metrics=compute_metrics,
    args=transformers.TrainingArguments(
        do_eval=True,
        output_dir=output_dir,
        warmup_ratio=0.05,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        num_train_epochs = 3,
        learning_rate=2e-4,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_strategy='epoch',
        report_to="wandb",
        run_name=wandb_run_name,
        # save_strategy="steps",
        # save_steps=25,

    ),
    max_seq_length=MAX_LEN,
    formatting_func=formatting_prompts_func, # 프롬프트 처리하기 위해 필요
    data_collator=DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer, mlm=False), #모델에게 답변생성에만 집중하도록 함
)

model.config.use_cache = False
trainer.train()

trained_model = (trainer.model.module if hasattr(trainer.model, "module") else trainer.model)

#로컬에 저장할 경우
# trained_model.save_pretrained(f"{output_dir}/saved_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/496 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
124,0.756600
248,0.643700
372,0.611400


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [21]:
train_name

'kdt3/DACON-QA-large-ensemble-markdown-newprompt-512'

In [22]:
train_name = train_name+'-epoch3ㅇㄷ-new'

In [23]:
# 모델 업로드
trained_model.push_to_hub(train_name, private=True,save_embedding_layers=True)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kdt3/DACON-QA-large-ensemble-markdown-newprompt-512-epoch3-new/commit/de695c05b55d8d2c67edc5f3edd3c7b9fb760eb4', commit_message='Upload model', commit_description='', oid='de695c05b55d8d2c67edc5f3edd3c7b9fb760eb4', pr_url=None, pr_revision=None, pr_num=None)

# Langchain 을 이용한 추론

In [24]:
# gpu memory 할당 해제
import gc, time

def free_cuda():
  mem = 1
  while mem > 0 :
    time.sleep(0.5)
    torch.cuda.empty_cache()
    mem = gc.collect()
    print("freed : ",mem)

free_cuda()

freed :  26
freed :  0


In [25]:
# HuggingFacePipeline 객체 생성

# 모델 ID
model_id = model_cands[run_name.split('_')[0]]
peft_model_id = train_name
trained_model,tokenizer = load_model_w_setting(model_id)

#Fine-Tune 한 LoRA 어댑터 불러오기
trained_model.load_adapter(peft_model_id)

text_generation_pipeline = pipeline(
    model=trained_model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    max_new_tokens=200,
    # repetition_penalty=1.5,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.pad_token_id,
    max_length=MAX_LEN
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:498: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

<ipython-input-25-07d937d7adb2>:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [26]:
from datasets import load_dataset
dataset_url = dataset_name
reload = 0

# 검증 데이터 쓸지, 테스트 데이터 쓸지
#데이터셋 로드
#eval_mode = 'valid'
eval_mode = 'test'

if reload :
  dataset = load_dataset(dataset_url)
else : dataset = train_dataset

eval_dataset = dataset[eval_mode]

In [27]:
# 그냥 GPU 메모리 확인용
!nvidia-smi

Mon Oct  7 10:05:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              59W / 400W |  15193MiB / 40960MiB |     14%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [28]:
end_idx = -13
inf_template = template[:end_idx]#+"<eos>"
inf_template

'"task_instructions" : [\n\n당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 세 문장 이내로 답변하세요.\n 1. 문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.\n 2. 순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.\n 3. 수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.\n 4. 질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.\n 5. 한 단어 혹은 단어의 나열이 아닌, 완성된 한국어 문장들로 답변하세요.\n 6. 최대한 간략한 내용으로, 최대한 적은 개수의 문장으로 답변하세요.\n 7. 답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, \'답변: \', \'참고: \'를 절대로 출력하지 마세요.\n\n]\n\n"context":\n{context},\n\n"question":\n{question},\n\n"주어진 질문에 대한 답변만 한 문장으로 생성한다."\n\n"answer":\n'

In [29]:
from tqdm.auto import trange

# 결과를 저장할 리스트 초기화
results = []

# DATASET 구조를 dataset[i]={'question':,'context':,...}로 바꾸면 안됨?

# Dataset 각 행에 대해 처리
for idx in trange(len(eval_dataset['question']), desc="Answering Questions"):
    #질문, 컨텍스트(문서)
    question = eval_dataset['question'][idx]
    context = eval_dataset['context'][idx]

    # RAG 체인 구성
    prompt = PromptTemplate.from_template(inf_template)

    # RAG 체인 정의
    if context != "":
        rag_chain = (
            RunnableParallel(context=lambda x: x["context"], question = lambda x: x["question"])
            | prompt
            | llm
            | StrOutputParser()
        )
    else:
        rag_chain = (
            {"question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke({"question": question, "context": context})

    print(f"Answer: {full_response}\n")

    # 결과 저장

    if context != "":
        results.append({
            "Context": context,
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })
    else:
        results.append({
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=200) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Both `max_new_tokens` (=200) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다.

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


...

Answer: 미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급여의 현재가치에서 현행법과 규정에 따른 사회보장제도의 미래보험료의 순현재가치를 차감한 것을 인식한다.

Question: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?


Both `max_new_tokens` (=200) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 국가결산보고서는 우발부채를 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준에서는 '우발손실'로 정의하고 있습니다.

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Both `max_new_tokens` (=200) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer: 우발부채는 미래의 의제의무를 포괄하는 재정적 책임으로, 재정건전성과 재정위기 관리 등 다양한 관점에서 중요하게 관리되어야 한다.

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?
Answer: 보증은 명시적 우발부채의 가장 보편적 형태이며, 파생금융상품 형태의 보증과 표준화 보증은 재정상태표에 부채로 기록되며, 표준화 보증은 수출신용보증, 환보증, 보험, 농민융자, 모기지론, 학자금융자, 중소기업융자 등 유사한 특징을 지니고 위험 분산을 목적으로 발행된다.



In [31]:
i = 20
print_dataset_ele(eval_dataset,i)

장기요양인정점수 완화가 언제 이루어졌으며, 어떤 변화가 있었나?
--------
-  (’12.9월) “제1차 장기요양 기본계획” 수립ㆍ발표  
-  (’13.7월) 노인장기요양보험 대상자 확대 (장기요양인정점수 완화)  
-  3등급 인정점수 완화: (기존) 53∼75점 → (개선) 51점∼75점  
-  (’14.7월) 노인장기요양보험 등급체계 개편(3등급 → 3등급 및 4등급으로 세분화)  
및 치매특별등급(5등급) 신설 관련 시행령ㆍ시행규칙ㆍ고시 개정
3. 직장가입자가 교직원(사립학교에 근무하는 교원은 제외한다)인 경우에는 제3조제2호다목에 해당하는 사용자  
② 추진경위  
-  (’01.8.15.) 고령화시대 대비, 노인장기요양보장제도 도입 발표  
-  (’07.4월)「노인장기요양보험법」제정ㆍ공포 (’08.7월부터 제도 시행)  
-  (’12.7월) 노인장기요양보험 대상자 확대 (장기요양인정점수 완화)  
-  3등급 인정점수 완화: (기존) 55∼75점 → (개선) 53점∼75점
-  (’16.5월) 장기요양기관 투명성 확보 마련 등을 위한 장기요양기관 재무·회계기준  
개정(’16.5.) 및 관련 시행규칙 시행(’18.5.)  
-  (’18.1월) 경증치매환자 “인지지원등급” 신설  
-  (’18.2월) “제2차 장기요양 기본계획” 수립ㆍ발표  
-  (’18.7월) 장기요양 보장성 강화를 위한 본인부담 감경대상 및 감경율 확대
# 5. 사업근거 및 추진경위  
① 법령상 근거 및 조항 : 노인장기요양보험법 제4조, 제11조, 제35조의2, 제58조 및  
같은 법 시행령제28조, 국민건강보험법 제76조  
노인장기요양보험법 제4조(국가 및 지방자치단체의 책무 등) ④ 국가 및 지방자치단체는 장기요양  
급여가 원활히 제공될 수 있도록 공단에 필요한 행정적 또는 재정적 지원을 할 수 있다.
-  (기존) 기준중위소득 50% 이하 대상 본인부담 50% 감경 → (변경) 건보료 기준 하위 25% 대상  
60% 감경, 하위 25~50% 대상 

In [30]:
def post_process(ans):
  ans = ans.strip().split('\n')
  sen_len = list(map(len,ans))
  th = 1
  for i,l in enumerate(sen_len):
    if l < 2 :
      th = i
      break
  if th < 1 : th = 1
  return '\n'.join(ans[:th])

In [31]:
def post_process_basic(ans):
  ans = ans.strip()
  return ans.split('\n')[0]

# Evaluation

In [32]:
rslt_df = pd.DataFrame(results)
rslt_df['Answer_post'] = rslt_df['Answer'].apply(post_process)

if eval_mode == 'valid':
  temp0 = rslt_df[['Answer','True_Answer']].apply(lambda x : compute_f1(*x),axis=1)
  temp1 = rslt_df[['Answer_post','True_Answer']].apply(lambda x : compute_f1(*x),axis=1)
  display(temp0)
  rslt_df[['F1_0','Precision_0','Recall_0']] = pd.DataFrame(temp0.tolist(), index= temp0.index)
  rslt_df[['F1_1','Precision_1','Recall_1']] = pd.DataFrame(temp1.tolist(), index= temp1.index)

rslt_df

,Context,Question,Answer,True_Answer,Answer_post
0,- 2014. 1 지원대상 업력기준을 7년 미만으로 확대 \n- 2015. 1...,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다.",,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다."
1,## 4. 사업목적 \nㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가...,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?,중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의 제품화·사업화를 촉진하여 ...,,중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의 제품화·사업화를 촉진하여 ...
2,## 6. 주요내용 \nㅇ총사업비 : 해당없음 \nㅇ 사업기간 : ’98년~ 계...,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제35조...",,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제35조..."
3,- 2014. 1 지원대상 업력기준을 7년 미만으로 확대 \n- 2015. 1...,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?,2010년에는 재창업자금(실패 경영인에 대한 재기지원)이 신규로 지원되었습니다.,,2010년에는 재창업자금(실패 경영인에 대한 재기지원)이 신규로 지원되었습니다.
4,- 2014. 1 지원대상 업력기준을 7년 미만으로 확대 \n- 2015. 1...,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?,2020년에 미래기술육성자금과 고성장촉진자금이 신규로 지원되었습니다.,,2020년에 미래기술육성자금과 고성장촉진자금이 신규로 지원되었습니다.
...,...,...,...,...,...
93,"|---|---| \n**자료: IMF, (2017), 「Government Fi...",재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...,,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,"다고 여겨 정부의 계약상 부채를 초래하지 않는 것으로 간주하며, 우발부채로 인식 ...",미래사회보장급여에 대한 순의무란 무엇을 의미하는가?,미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급...,,미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급...
95,"- 「국가회계기준에 관한 규칙」은 제50조에서 ‘충당부채’, ‘우발부채’ 및 ‘우...",국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...,"국가결산보고서는 우발부채를 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준에서...",,"국가결산보고서는 우발부채를 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준에서..."
96,## 들어가며 \n**ISSUE Col1** \n**왜 우발부채에 주목하는가?*...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?","우발부채는 미래의 의제의무를 포괄하는 재정적 책임으로, 재정건전성과 재정위기 관리 ...",,"우발부채는 미래의 의제의무를 포괄하는 재정적 책임으로, 재정건전성과 재정위기 관리 ..."


In [33]:
np.sum(rslt_df['Answer'] != rslt_df['Answer_post'])

0

In [34]:
if eval_mode == 'valid' :
  display("결과 : ",rslt_df["F1_0"].mean())
  display("결과 : ",rslt_df["F1_1"].mean())

In [35]:
if eval_mode == 'valid':
  # 제출용 샘플 파일 로드
  eval_df = rslt_df
  save_dir = os.path.join(path,'eval')
  if not os.path.exists(save_dir) : os.makedirs(save_dir)
  save_name = f'eval_{fname}'
  save_path = os.path.join(save_dir,save_name)

  # 결과를 CSV 파일로 저장
  eval_df.to_csv(save_path, encoding='UTF-8-sig', index=False)

In [36]:
#검증 데이터 쓸때만 사용 가능
if eval_mode == 'valid':
  for item in results:
      y_hat = item["Answer"]
      y = item["True_Answer"]
      f1, precision, recall = compute_f1(y, y_hat)
      item["F1"] = f1
      item["Precision"] = precision
      item["Recall"] = recall

# Submission

In [37]:
submit_df = pd.read_csv(f"{path}sample_submission.csv")
submit_df.head()

,SAMPLE_ID,Answer
0,TEST_000,데이콘
1,TEST_001,데이콘
2,TEST_002,데이콘
3,TEST_003,데이콘
4,TEST_004,데이콘


In [38]:
fname

'gemma2_large_ensemble_markdown_512_3epoch_refactored.csv'

In [40]:
fname = 'gemma2_large_ensemble_markdown_512_3epoch_newprompt.csv'

In [41]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(f"{path}sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
save_mode = 'submission'

if save_mode != 'submission' :
  submit_df['Question'] = [item['Question'] for item in results]
  submit_df['Context'] = [item['Context'] for item in results]
  save_dir = os.path.join(path,'eval')
else : save_dir = os.path.join(path,'sub')

if not os.path.exists(save_dir) : os.makedirs(save_dir)
save_path1 = os.path.join(save_dir,fname[:-4]+'_1.csv')
save_path2 = os.path.join(save_dir,fname[:-4]+'_2.csv')

submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘").apply(str.strip)

# 결과를 CSV 파일로 저장
submit_df.to_csv(save_path1, encoding='UTF-8-sig', index=False)

# 후처리
#submit_df['Answer'] = submit_df.Answer.str.split('\n').str[0]
submit_df['Answer'] = submit_df.Answer.apply(post_process)
display(submit_df)
submit_df.to_csv(save_path2, encoding='UTF-8-sig', index=False)

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다."
1,TEST_001,중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의 제품화·사업화를 촉진하여 ...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제35조..."
3,TEST_003,2010년에는 재창업자금(실패 경영인에 대한 재기지원)이 신규로 지원되었습니다.
4,TEST_004,2020년에 미래기술육성자금과 고성장촉진자금이 신규로 지원되었습니다.
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급...
95,TEST_095,"국가결산보고서는 우발부채를 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준에서..."
96,TEST_096,"우발부채는 미래의 의제의무를 포괄하는 재정적 책임으로, 재정건전성과 재정위기 관리 ..."


In [ ]:
os.listdir(save_dir)

['gemma2_large_ensemble_markdown_256_3epoch_refactored.csv',
 'gemma2_large_ensemble_markdown_256_3epoch_refactored_1.csv',
 'gemma2_large_ensemble_markdown_256_3epoch_refactored_2.csv',
 'gemma2_large_ensemble_markdown_256_6epoch_refactored_1.csv',
 'gemma2_large_ensemble_markdown_256_6epoch_refactored_2.csv',
 'gemma2_large_ensemble_markdown_256_5epoch_refactored_1.csv',
 'gemma2_large_ensemble_markdown_256_5epoch_refactored_2.csv']

In [ ]:
old_path = os.path.join(save_dir,'gemma2_large_ensemble_markdown_256_6epoch_refactored_2.csv')
old_submit = pd.read_csv(old_path)
old_submit

,SAMPLE_ID,Answer
0,TEST_000,"2022년 혁신창업사업화자금(융자)의 예산은 2,330,000,000원입니다."
1,TEST_001,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 중소기업이 보유한 우수 기술의 ...
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제35조..."
3,TEST_003,2010년에 재창업자금(실패 경영인에 대한 재기지원)이 신규 지원되었습니다.
4,TEST_004,2020년에는 미래기술육성자금과 고성장촉진자금이 신규로 지원되었습니다.
...,...,...
93,TEST_093,공적보증채무는 보증인이 기타 공공부문과 민간부문 단위 기존 채무의 원리금 상환을 보...
94,TEST_094,미래사회보장급여에 대한 순의무는 현행법과 규정에 따라 이미 가입자가 획득한 미래 급...
95,TEST_095,"국가결산보고서는 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준은 '우발손실'..."
96,TEST_096,"우발부채는 미래의 다양한 의제의무를 포괄하는 재정위험을 발생시키는 구체적 원인으로,..."


In [ ]:
cond = old_submit['Answer']!=submit_df['Answer']
df_diff = pd.concat([old_submit[['SAMPLE_ID','Answer']],submit_df['Answer']],axis=1).loc[cond]
for i in range(len(df_diff))[15:]:
  idx = int(df_diff.iloc[i,0].split('_')[-1])
  print(idx)
  print_dataset_ele(eval_dataset,idx)
  print(df_diff.iloc[i,1])
  print(df_diff.iloc[i,2])
  print('-'*30)

77
재정관리시스템 구축과 성과관리 개편을 추진하는 주된 목적은 무엇인가?
--------
①성과관리의 내용을 성과목표관리 및 성과평가로 구체화  
②재정사업 성과관리 기본계획 수립, 성과보고서 작성 및 성과평가 근거 마련  
③재정사업 성과목표 관리를 위한 추진체계 마련  
④성과평가 결과 등을 저장, 활용할 수 있는 성과정보관리시스템 구축·운영  
⑤성과목표 관리 결과를 국무회의에 보고하여 성과 중심 재정운용 확대 및 강화  
‣ 재정성과관리제도 개선으로 인하여 대폭 성과관리의 효과성이 강화될 것으로 전망
## 나가며  
**정책적 함의**  
‣ 재정운용의 효율성, 책임성 제고를 위한 재정성과관리제도의 개선은 계속되고 있음  
-  현 정부는 재정비전2050 수립을 위한 추진 과제로 성과관리 개편 및 재정관리시스템 구축 등을 추진하여 지  
속적으로 제도 개선, 인프라 개선을 해나갈 전망  
-  재정관리는 어느 한 시점에서 체제를 완비한다고 해서 되는 것이 아니며, 지속적인 제도 개선을 통해 변화하  
는 재정 환경에 적응해 나갈 수 있어야 함
의 기본 단위, 성과 측정과 보고를 위한 체계 확립  
-  전 부처 재정사업에 대해 기관 전략목표-성과지표에 의한 부처의 자율적 성과관리, 재정당국의 감독 및 심층  
평가라는 현행 재정사업 관리체계를 구축하여 운영하고 있음  
-  이 과정에서 생산되는 정보를 정보화하여 활용할 수 있는 기반을 마련  
-  이로 인하여 성과 중심 운영을 위한 제도를 성문화하고, 제반 인프라를 제공할 수 있게 되어 공개를 통한 투명
을 제고할 수 있음  
**재정성과관리 관련 주요 쟁점은?**  
**FOCUS Col1**  
-  「국가재정법」상의 재정사업 평가인 재정사업 자율평가의 대상, 범위, 효과, 개편 등은 지속적으로 논의되어야  
하며, 개별 법령에 따라 실시되는 평가 대상 간 중복 최소화 노력 필요  
-  재정성과관리 결과를 재정 운용에 반영하고, 또 국민에게 알기 쉽게 홍보할 수 있도록 공개하기 위한 지속적  

In [ ]:
idx_list = [16,49,59,74,77,80]

In [ ]:
for idx in idx_list:
  print(idx)
  print_dataset_ele(eval_dataset,idx)
  print("<eos> 없음 :",df_diff.loc[idx].iloc[1])
  print("<eos> 추가 :",df_diff.loc[idx].iloc[2])
  print('-'*30)

16
노인장기요양보험 운영지원에 대한 사업 내용을 설명해줘.
--------
# 4. 사업목적·내용  
-  (노인장기요양보험 사업운영) 고령이나 노인성 질병으로 일상생활을 혼자서 수행하기  
어려운 노인등에게 신체 또는 가사 활동 등을 제공하는 노인장기요양보험에 국고지원을  
하여, 효율적인 정책추진으로 노후의 건강증진 및 생활 안정을 도모하고 가족의 부담을  
완화하여 국민 삶의 질을 향상  
-  (노인장기요양보험 운영지원) 「노인장기요양보험법」제58조에 따라 국가가 국민건강보험
"가입자"는 "장기요양보험가입자"로 본다.  
노인장기요양보험법 제35조의2(장기요양기관 재무·회계기준) ① 장기요양기관의 장은 보건복지부령으로 정  
하는 재무·회계에 관한 기준(이하 “장기요양기관 재무·회계기준”이라 한다)에 따라 장기요양기관을 투명  
하게 운영하여야 한다. 다만, 장기요양기관 중 「사회복지사업법」제34조에 따라 설치한 사회복지시설은  
같은 조 제3항에 따른 재무ㆍ회계에 관한 기준에 따른다.
# 사  업  명  
(74) 노인장기요양보험 사업운영 (2231-303)
함한다) 및 관리운영비의 전액을 부담한다.  
노인장기요양보험법 시행령 제28조(국가와 지방자치단체의 부담) ① 법 제58조제2항에 따른 의  
료급여수급권자에 대한 국가와 지방자치단체의 비용 부담은 다음 각 호의 기준에 따른다.  
1의 2. 「의료급여법」제3조제1항제1호에 따른 의료급여를 받는 사람에 대한 비용 : 지방자치단체가 부담한다.  
2. 「의료급여법」제3조제1항제1호 외의 규정에 따른 의료급여를 받는 사람에 대한 비용 : 각 목의
# 5. 사업근거 및 추진경위  
① 법령상 근거 및 조항 : 노인장기요양보험법 제4조, 제11조, 제35조의2, 제58조 및  
같은 법 시행령제28조, 국민건강보험법 제76조  
노인장기요양보험법 제4조(국가 및 지방자치단체의 책무 등) ④ 국가 및 지방자치단체는 장기요양  
급여가 원활히 제공될 수 있도록 공단에 필요한 행정적 또는 재정적 지원을 할